In [1]:
import datetime
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Preprocessing

Goal: Pre-process 2 datasets and 

In [2]:
train_actress = pd.read_json("input/actress.json")
train_movie = pd.read_csv("input/movies.csv")

In [3]:
train_actress.head()

,birthday,bust,height,hip,hobby,id,imageUrl,japanName,name,prefectures,siteUrl,waist
0,1980-12-13T16:00:00.000Z,95,0,88,クラシックバレエ,3,https://pics.dmm.co.jp/mono/actjpgs/kitagawa_y...,北川ゆい,Kitagawa Yui,,http://www.dmm.co.jp/digital/videoa/-/list/=/a...,60
1,1978-08-14T16:00:00.000Z,90,157,86,水泳、ゴルフ,4,https://pics.dmm.co.jp/mono/actjpgs/akira.jpg,Akira,Akira,東京都,http://www.dmm.co.jp/digital/videoa/-/list/=/a...,58
2,1982-07-06T16:00:00.000Z,96,168,85,パソコン,6,https://pics.dmm.co.jp/mono/actjpgs/cocolo.jpg,COCOLO,Cocolo,茨城県,http://www.dmm.co.jp/digital/videoa/-/list/=/a...,55
3,1983-04-11T16:00:00.000Z,80,162,82,料理、大食い,8,https://pics.dmm.co.jp/mono/actjpgs/saiko.jpg,Saiko,Saiko,東京都,http://www.dmm.co.jp/digital/videoa/-/list/=/a...,59
4,1970-06-17T16:00:00.000Z,82,162,89,エアロビクス,11,https://pics.dmm.co.jp/mono/actjpgs/aida_momo.jpg,あいだもも,Aida Momo,東京都,http://www.dmm.co.jp/digital/videoa/-/list/=/a...,55


In [4]:
train_movie.head()

,actress,genre,maker,imageUrl,name,price,length,actress_id
0,"405,奥菜千春","顔射,単体作品,ローター,学生服,OL,レズ,コスプレ,ベスト・総集編,女子校生,バイブ,中出し","その他,other,（TQT）",{'list': 'http://pics.dmm.co.jp/digital/video/...,特選 奥菜千春,300,120,405
1,"1251,手塚美紗","単体作品,P・P,コスプレ,ベスト・総集編,バイブ,中出し","その他,other,（TQT）",{'list': 'http://pics.dmm.co.jp/digital/video/...,特選 手塚美紗,300,120,1251
2,"1956,川浜理奈","潮吹き,単体作品,P・P,パイパン,ぶっかけ","その他,other,ankylosaur",{'list': 'http://pics.dmm.co.jp/digital/video/...,潮吹剃毛 to shave！ 川浜理奈,300,45,1956
3,"2021,早川あゆみ","単体作品,縛り・緊縛,ごっくん,P・P,パイパン,バイブ","その他,other,ankylosaur",{'list': 'http://pics.dmm.co.jp/digital/video/...,強制剃毛 to shave！ 早川あゆみ,300,45,2021
4,"2472,藤丸らん","単体作品,ごっくん,P・P,パイパン,バイブ","その他,other,ankylosaur",{'list': 'http://pics.dmm.co.jp/digital/video/...,剃毛体験 to shave！ 藤丸らん,300,45,2472


Counting number of movies per actress by using movie data

In [5]:
def _counting_movies(actress_id):
    return len(train_movie[train_movie.actress_id == actress_id])
train_actress['no_movies'] = train_actress['id'].apply(lambda x: _counting_movies(x))

Remove NAN values

In [6]:
train_actress = train_actress[train_actress.no_movies != 0]
train_actress = train_actress[train_actress['birthday'].notnull()]

Calculate the Age

In [7]:
train_actress['birthday'] = pd.to_datetime(train_actress['birthday'])
train_actress['age'] = train_actress['birthday'].apply(lambda x: pd.to_datetime('today').year - x.year)

Counting number of genres

In [17]:
def _counting_genre(actress_id):
    actress_info = train_movie[train_movie.actress_id == actress_id]
    return len(list(set(','.join(actress_info['genre']).split(','))))
train_actress['no_genres'] = train_actress['id'].apply(lambda x: _counting_genre(x))

In [21]:
train_movie[train_movie.actress_id == 8]

,actress,genre,maker,imageUrl,name,price,length,actress_id
13312,"8,Saiko","単体作品,デビュー作品,イラマチオ,フェラ,ぶっかけ,独占配信,企画",ムーディーズ,{'list': 'http://pics.dmm.co.jp/digital/video/...,プロジェクトSEX エロ戦士たち Saiko,200,110,8


In [25]:
final_dataset = train_actress[['id', 'name', 'bust', 'waist', 'hip', 'birthday', 'age', 'hobby', 'prefectures', 'no_movies', 'no_genres', 'imageUrl']]

In [26]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3664 entries, 0 to 10156
Data columns (total 12 columns):
id             3664 non-null int64
name           3664 non-null object
bust           3664 non-null int64
waist          3664 non-null int64
hip            3664 non-null int64
birthday       3664 non-null datetime64[ns]
age            3664 non-null int64
hobby          2856 non-null object
prefectures    2712 non-null object
no_movies      3664 non-null int64
no_genres      3664 non-null int64
imageUrl       3664 non-null object
dtypes: datetime64[ns](1), int64(7), object(4)
memory usage: 372.1+ KB


In [27]:
final_dataset.describe()

,id,bust,waist,hip,age,no_movies,no_genres
count,3.664000e+03,3664.000000,3664.000000,3664.000000,3664.000000,3664.000000,3664.000000
mean,3.014347e+05,84.686135,57.439683,84.317959,33.874181,25.974618,32.987718
std,4.569054e+05,13.234617,7.570916,11.169952,6.893935,57.496261,28.410337
min,3.000000e+00,0.000000,0.000000,0.000000,20.000000,1.000000,1.000000
25%,4.243500e+03,83.000000,57.000000,84.000000,29.000000,4.000000,12.750000
50%,1.590200e+04,85.000000,58.000000,86.000000,33.000000,9.000000,23.000000
75%,1.004165e+06,88.000000,60.000000,88.000000,38.000000,24.000000,44.000000
max,1.042220e+06,124.000000,87.000000,100.000000,63.000000,1090.000000,181.000000


In [28]:
final_dataset = final_dataset.sort_values(by="bust")

In [30]:
final_dataset.tail()

,id,name,bust,waist,hip,birthday,age,hobby,prefectures,no_movies,no_genres,imageUrl
5887,29790,Matuzaka Minami,115,58,85,1984-02-06 16:00:00,34,ベリーダンス、ホットヨガ、ピアノ、料理、アロママッサージ、日本舞踊（藤間流）、芝居、ボクシング,千葉県,1,5,https://pics.dmm.co.jp/mono/actjpgs/matuzaka_m...
7233,1014982,Sakuragi Ria,115,59,85,1990-09-10 16:00:00,28,スポーツ,,40,45,https://pics.dmm.co.jp/mono/actjpgs/sakuragi_r...
6416,1005536,Sakura Yuki,115,68,94,1990-04-29 16:00:00,28,ネイル、ダーツ、カラオケ,,3,7,https://pics.dmm.co.jp/mono/actjpgs/sakura_yuk...
5234,23933,Huuko,120,60,88,1988-02-09 16:00:00,30,胸を使った一発芸、ソフトボール、映画鑑賞、食べる事、カラオケ,,12,16,https://pics.dmm.co.jp/mono/actjpgs/huuko.jpg
3472,10991,Jyou Eren,124,65,90,1971-07-14 16:00:00,47,歌・ピアノ・エレクトーン・作曲・格闘技観戦・旅行・天体観測・ガーデニング,神奈川県,47,46,https://pics.dmm.co.jp/mono/actjpgs/jyou_eren.jpg


### Process unicode columns

In [ ]:
def _binary_checking(key, string):
    if key in string:
        return 1
    return 0

Process the Genre

In [ ]:
def genre_finding(actress_id, genre):
    if len(train_movie[train_movie.actress_id == actress_id]) > 0:
        genre_string = ','.join(train_movie[train_movie.actress_id == actress_id]['genre'])
        return _binary_checking(genre, genre_string)

In [15]:
tmp_list = list(set(','.join(train_movie['genre']).split(',')))

In [16]:
len(tmp_list)

256

In [ ]:
for genre in tmp_list:
    train_actress[genre] = train_actress['id'].apply(lambda x: genre_finding(x, genre))

Process the maker

In [ ]:
def maker_finding(actress_id, maker):
    if len(train_movie[train_movie.actress_id == actress_id]) > 0:
        maker_string = ','.join(train_movie[train_movie.actress_id == actress_id]['maker'])
        return _binary_checking(maker, maker_string)

In [ ]:
maker_list = list(set(','.join(train_movie['maker']).split(',')))

In [ ]:
train_actress.to_csv("output/train_actress.csv", sep='\t', encoding='utf-8')

In [ ]:
for maker in maker_list:
    train_actress[str(maker) + '_maker'] = train_actress['id'].apply(lambda x: maker_finding(x, maker))

In [ ]:
train_actress.to_csv("output/train_actress_full.csv", sep='\t', encoding='utf-8')

In [ ]:
train_actress = pd.read_csv("output/train_actress_full.csv", sep='\t', encoding='utf-8')

In [ ]:
train_actress.info()